# DSCI 100 - Course Project Report

Title: Predicting Credit Card Default

Group 10: Berkay Talha Acar, Stephen Pon

# Introduction 

The dataset we will use is called “default of credit card clients”. It is credit card data collected from Taiwan by Chung Hua University, Taiwan. A credit default is when a client fails to repay their debt, interest or principal, on a credit card loan. A default can occur when a borrower is unable to make timely payments, misses payments, or avoids/stops making payments. Default risks are often calculated well in advance by creditors such as banks who set a limit on the clients’ debt. 

In this project, we will build a K-nearest neighbors classification model that can predict whether a subset of clients with a university education will default based on various credit risk metrics.

# Methods and Results 

After the appropriate libraries are downloaded, the dataset can be read from the web into R as follows:

In [1]:
library(tidyverse)
library(readxl)
library(testthat)
library(digest)
library(repr)
library(tidymodels)
library(cowplot)
#library(GGally)
library(ISLR)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘testthat’


The following object is masked from ‘package:dplyr’:

    matches


The following object is masked from ‘package:purrr’:

    is_null


The following object is masked from ‘package:tidyr’:

    matches


Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────

In [3]:
set.seed(19) # set the seed for reproducibility
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
download.file(url, "credit_default.xls")
cd <- read_excel("credit_default.xls", skip = 1)
head(cd)
dim(cd)

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,⋯,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,20000,2,2,1,24,2,2,-1,-1,⋯,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,⋯,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,⋯,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,⋯,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,⋯,20940,19146,19131,2000,36681,10000,9000,689,679,0
6,50000,1,1,2,37,0,0,0,0,⋯,19394,19619,20024,2500,1815,657,1000,1000,800,0


[1] 30000    25

We see that our dataset is 30,000 by 25, and to clean our dataset, we will first remove the spaces from column names and filter for people with university degree.

In [4]:
colnames(cd) <- make.names(colnames(cd)) # remove the spaces from column names 
cd_university <- cd %>%
                filter(EDUCATION == 2) # university education
head(cd_university)
dim(cd_university) # let's see the dimensions after filtered for cd_university, which is 14030x25

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,⋯,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,20000,2,2,1,24,2,2,-1,-1,⋯,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,⋯,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,⋯,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,⋯,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,⋯,20940,19146,19131,2000,36681,10000,9000,689,679,0
8,100000,2,2,2,23,0,-1,-1,0,⋯,221,-159,567,380,601,0,581,1687,1542,0


[1] 14030    25

Next, we randomly sample 10,000 rows from our dataset to avoid spending a lot of time training our model, or crashing the server with too many observations, and to eliminate bias that might exist in our initial dataset. We also think that it is valuable to have enough data in training, so that our model can learn and generalize from examples.

In [7]:
#sample 10,000 random rows since load times get very long with too much data
cd_sample <- sample_n(cd_university, 10000) %>% # sample 10,000 rows
        rename(default_nm = default.payment.next.month, # rename the classification variable 
                limit = LIMIT_BAL) %>%
        mutate(default_nm = as_factor(default_nm), # factor our classsification variable 
              bill_avg = ((BILL_AMT1 + BILL_AMT2 + BILL_AMT3 + BILL_AMT4 + BILL_AMT5 + BILL_AMT6)/6), # average of bills
              pay_avg = ((PAY_AMT1 + PAY_AMT2 + PAY_AMT3 + PAY_AMT4 + PAY_AMT5 + PAY_AMT6)/6)) # average of payments
head(cd_sample)
tail(cd_sample)
dim(cd_sample) # our initial data frame with 10,000 rows and  27 columns

ID,limit,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,⋯,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_nm,bill_avg,pay_avg
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>
13722,60000,1,2,2,38,0,0,0,0,⋯,39737,2014,1616,1566,1700,1600,1400,0,41089.1667,1649.3333
15000,50000,1,2,2,28,0,0,0,0,⋯,43863,1667,1987,1655,1710,1719,1616,0,41618.5000,1725.6667
19834,210000,2,2,1,40,0,0,0,0,⋯,104167,4100,5000,4009,3500,4000,5000,0,110816.0000,4268.1667
10120,90000,2,2,2,26,0,0,0,0,⋯,73940,3300,3168,3143,2700,3600,1700,0,72520.8333,2935.1667
11694,230000,2,2,1,32,-2,-2,-2,-2,⋯,2803,0,0,790,0,2803,0,0,723.8333,598.8333
15576,110000,1,2,2,55,0,0,0,0,⋯,111719,65668,4267,9455,0,3991,8071,1,102567.8333,15242.0000


ID,limit,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,⋯,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_nm,bill_avg,pay_avg
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>
15614,120000,2,2,1,30,0,0,0,0,⋯,122094,4700,4400,5000,4600,5000,4500,0,121211.667,4700.0000
10630,210000,1,2,2,33,-1,-1,-1,-1,⋯,43104,264,264,264,42515,1559,1472,0,14572.500,7723.0000
10627,390000,1,2,1,35,0,-1,-1,0,⋯,57954,15347,17375,70077,15103,3290,16153,0,46600.000,22890.8333
5960,50000,2,2,2,24,1,2,0,0,⋯,19929,0,1636,700,1500,1000,500,0,22903.167,889.3333
21533,10000,2,2,2,23,1,2,3,3,⋯,7990,2000,1000,0,0,500,900,1,7465.000,733.3333
26409,240000,2,2,2,36,-1,-1,-1,-1,⋯,0,2947,297,0,0,0,0,1,1194.333,540.6667


[1] 10000    27

We split our data into training (75%) and testing (25%) datasets. 75% of the dataset is chosen for training, because we need much more data for training, and 25% is more than enough for testing our model. We will only use the training set for our exploratory analysis.

In [8]:
cd_split <- initial_split(cd_sample, prop=0.75, strata=default_nm)
cd_training <- training(cd_split)
cd_testing <- testing(cd_split)

Here, we create our variables of interest. `usage_cdt` and `pmt` represent, on average, the client's bills and payments as a proportion of their available credit card limit.

In [9]:
cd_training <- cd_training %>%
        mutate(payment_to_bill_ratio = pay_avg/bill_avg) %>%  # the ratio of their average payment to average bill    
        mutate(usage_cdt = bill_avg/limit) %>% # to see how much of their balance they use 
        mutate(pmt = pay_avg/limit) %>% # to see how much of their balance they pay
        mutate(diff_usage_pmt = usage_cdt - pmt) %>%
        mutate(pay_overall = PAY_0 + PAY_2 + PAY_3 + PAY_4 + PAY_5 + PAY_6) # to see how many months they are behind in 6 months period, if x positive means you are behind payment, if x negative means pay dully   
head(cd_training)

ID,limit,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,⋯,PAY_AMT5,PAY_AMT6,default_nm,bill_avg,pay_avg,payment_to_bill_ratio,usage_cdt,pmt,diff_usage_pmt,pay_overall
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
13722,60000,1,2,2,38,0,0,0,0,⋯,1600,1400,0,41089.167,1649.3333,0.04014035,0.6848194,0.02748889,0.6573306,0
19834,210000,2,2,1,40,0,0,0,0,⋯,4000,5000,0,110816.000,4268.1667,0.03851580,0.5276952,0.02032460,0.5073706,0
10120,90000,2,2,2,26,0,0,0,0,⋯,3600,1700,0,72520.833,2935.1667,0.04047343,0.8057870,0.03261296,0.7731741,4
15576,110000,1,2,2,55,0,0,0,0,⋯,3991,8071,1,102567.833,15242.0000,0.14860409,0.9324348,0.13856364,0.7938712,2
7755,20000,1,2,2,22,0,0,-2,-2,⋯,601,0,0,2977.833,773.8333,0.25986455,0.1488917,0.03869167,0.1102000,-6
1635,170000,2,2,1,50,0,0,0,0,⋯,39321,1390,0,115828.833,9804.8333,0.08464933,0.6813461,0.05767549,0.6236706,0


The table below is created depending on default status where 1 represents default and 0 indicates non-default status. `mean_pay0` represents the average number of payments that are owed or have been paid for the two groups in the last period of available data. Negative values indicate the average client is ahead on payments and positive indicates they are behind. 

`mean_diff_usage_pmt` measures the average difference between the ratio of bills to limit and payments to limit. In theory, the greater the difference, the more an individual is spending and the less they are paying. Notice that credit limit for the non-default group is significantly larger but the average bill only slightly larger compared to those who default. This means those in the default group use, on average, a larger proportion of their available credit.

In [13]:
cd_training_means <- cd_training %>%
                group_by(default_nm) %>%
                summarize(n = n(),
                         mean_pay0 = mean(PAY_0, na.rm = TRUE), # to see the means for PAY_0
                         mean_limit = mean(limit, na.rm = TRUE), # to see the means for limit
                         mean_bill = mean(bill_avg, na.rm = TRUE),
                         mean_pay = mean(pay_avg, na.rm = TRUE),
                         mean_usage_cdt = mean(usage_cdt, na.rm = TRUE),
                         mean_pmt = mean(pmt, na.rm = TRUE),
                         mean_diff_usage_pmt = mean(diff_usage_pmt, na.rm = TRUE),
                         mean_pay_overall = mean(pay_overall, na.rm = TRUE)
                         )
cd_training_means

`summarise()` ungrouping output (override with `.groups` argument)



default_nm,n,mean_pay0,mean_limit,mean_bill,mean_pay,mean_usage_cdt,mean_pmt,mean_diff_usage_pmt,mean_pay_overall
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,5704,-0.1127279,156730.4,47404.96,5194.716,0.4089096,0.04207490,0.3668347,-1.247370
1,1797,0.7929883,110690.0,44686.14,3065.971,0.5065064,0.03713626,0.4693702,2.935448


We can analyze some of the columns we created by selecting the variables of interest. 

In [14]:
cd_training_select <- cd_training %>%
                select(limit, PAY_0, bill_avg, pay_avg, usage_cdt, pmt, diff_usage_pmt, pay_overall, default_nm)
head(cd_training_select) 

limit,PAY_0,bill_avg,pay_avg,usage_cdt,pmt,diff_usage_pmt,pay_overall,default_nm
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
60000,0,41089.167,1649.3333,0.6848194,0.02748889,0.6573306,0,0
210000,0,110816.000,4268.1667,0.5276952,0.02032460,0.5073706,0,0
90000,0,72520.833,2935.1667,0.8057870,0.03261296,0.7731741,4,0
110000,0,102567.833,15242.0000,0.9324348,0.13856364,0.7938712,2,1
20000,0,2977.833,773.8333,0.1488917,0.03869167,0.1102000,-6,0
170000,0,115828.833,9804.8333,0.6813461,0.05767549,0.6236706,0,0


In [ ]:
usage_cdt = ((BILL_AMT1 + BILL_AMT2 + BILL_AMT3 + BILL_AMT4 + BILL_AMT5 + BILL_AMT6)/6)/limit, # column that measures average proportion of credit limit that is used over the time period
              pmt = ((PAY_AMT1 + PAY_AMT2 + PAY_AMT3 + PAY_AMT4 + PAY_AMT5 + PAY_AMT6)/6)/limit) %>% # column that measures average proportion of credit limit that is paid over the time period

select(-ID, -SEX, -EDUCATION, -MARRIAGE, -AGE)
head(cd_sample)
tail(cd_sample)

SPLIT TESTTT

In [8]:
cd_university_mutated <- cd_university %>%
        rename(default_nm = default.payment.next.month) %>%
        mutate(default_nm = as_factor(default_nm)) %>%
        mutate(pay_avg = (PAY_AMT1 + PAY_AMT2 + PAY_AMT3 + PAY_AMT4 + PAY_AMT5 + PAY_AMT6)/6) %>%
        mutate(bill_avg = (BILL_AMT1 + BILL_AMT2 + BILL_AMT3 + BILL_AMT4 + BILL_AMT5 + BILL_AMT6))

head(cd_university_mutated)

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,⋯,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_nm,pay_avg,bill_avg
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>
1,20000,2,2,1,24,2,2,-1,-1,⋯,0,0,689,0,0,0,0,1,114.8333,7704
2,120000,2,2,2,26,-1,2,0,0,⋯,3261,0,1000,1000,1000,0,2000,1,833.3333,17077
3,90000,2,2,2,34,0,0,0,0,⋯,15549,1518,1500,1000,1000,1000,5000,0,1836.3333,101653
4,50000,2,2,1,37,0,0,0,0,⋯,29547,2000,2019,1200,1100,1069,1000,0,1398.0000,231334
5,50000,1,2,1,57,-1,0,-1,0,⋯,19131,2000,36681,10000,9000,689,679,0,9841.5000,109339
8,100000,2,2,2,23,0,-1,-1,0,⋯,567,380,601,0,581,1687,1542,0,798.5000,13486


In [29]:
# cd_corr <- ggpairs(cd_university_mutated)
# cd_corr


In [9]:
cd_split <- initial_split(cd_university_mutated, prop=0.75, strata=default_nm)
cd_training <- training(cd_split)
cd_testing <- testing(cd_split)

In [22]:
cd_uni <- cd_training %>%
        mutate(payment_to_bill_ratio = pay_avg/bill_avg) %>%  # the ratio of their average payment to average bill    
        mutate(usage_cdt = bill_avg/LIMIT_BAL) %>% # to see how much of their balance they use 
        mutate(pmt = pay_avg/LIMIT_BAL) %>% # to see how much of their balance they pay
        mutate(diff_usage_pmt = usage_cdt - pmt) %>% # 
        select(LIMIT_BAL, PAY_0, bill_avg, pay_avg, usage_cdt, pmt, diff_usage_pmt, PAY_6, default_nm)
head(cd_uni)
#cd_uni

LIMIT_BAL,PAY_0,bill_avg,pay_avg,usage_cdt,pmt,diff_usage_pmt,PAY_6,default_nm
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
90000,0,101653,1836.3333,1.12947778,0.020403704,1.10907407,0,0
50000,-1,109339,9841.5000,2.18678000,0.196830000,1.98995000,0,0
100000,0,13486,798.5000,0.13486000,0.007985000,0.12687500,-1,0
630000,-1,41007,3895.0000,0.06509048,0.006182540,0.05890794,-1,0
120000,-1,1896,263.3333,0.01580000,0.002194444,0.01360556,-1,1
70000,2,265475,1835.0000,3.79250000,0.026214286,3.76628571,2,1


In [28]:
# options(repr.plot.width = 10, repr.plot.height = 8)
# cd_corr <- select(cd_uni, PAY_0, default_nm) %>% ggpairs()
# cd_corr


In [ ]:
#cd_corr 

In [15]:
cd_recipe <- recipe(default_nm ~ LIMIT_BAL + usage_cdt + pmt + PAY_0, data = cd_uni) %>%
step_scale(all_predictors()) %>%
step_center(all_predictors())

cd_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
set_engine("kknn") %>%
set_mode("classification")
#cd_recipe
cd_spec

K-Nearest Neighbor Model Specification (classification)

Main Arguments:
  neighbors = tune()
  weight_func = rectangular

Computational engine: kknn 


In [17]:
cd_vfold <- vfold_cv(cd_uni, v=5, strata= default_nm)

grids <- tibble(neighbors = seq(1, 50, 3))

cd_results <- workflow() %>%
            add_recipe(cd_recipe) %>%
            add_model(cd_spec) %>%
            tune_grid(resamples = cd_vfold, grid=grids) %>%
            collect_metrics()
cd_results

neighbors,.metric,.estimator,mean,n,std_err,.config
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,accuracy,binary,0.7201367,5,0.002425915,Model01
1,roc_auc,binary,0.6123526,5,0.002927261,Model01
4,accuracy,binary,0.7680321,5,0.002067549,Model02
4,roc_auc,binary,0.6965768,5,0.002940774,Model02
7,accuracy,binary,0.7949259,5,0.002572713,Model03
7,roc_auc,binary,0.7179940,5,0.006318236,Model03
10,accuracy,binary,0.7992027,5,0.002734419,Model04
10,roc_auc,binary,0.7294159,5,0.006192167,Model04
13,accuracy,binary,0.8055700,5,0.003130852,Model05


# Discussion 

# References 